# Downloading Players Data of Clash of Clans

Clash of Clans API: https://developer.clashofclans.com/#/account

To access data for your project, first register an account on the project's developer site. Following login, the next step involves creating an API key. This key allows for the retrieval of project data, facilitating the development of applications or analysis based on the project's information.

# GET /clans

Clash of Clans Documents: https://developer.clashofclans.com/#/documentation

## Search clans

Search all clans by name and/or filtering the results using various criteria. At least one filtering criteria must be defined and if name is used as part of search, it is required to be at least three characters long. It is not possible to specify ordering for results so clients should not rely on any specific ordering as that may change in future releases of the API.

## Parameters

| Name | Type | In | Description |
|------|------|----|----|
| name | string | query | Search clans by name. If name is used as part of search query, it needs to be at least three characters long. Name search parameter is interpreted as wild card search, so it may appear anywhere in the clan name. |
| warFrequency | string | query | Filter by clan war frequency |
| locationId | integer | query | Filter by clan location identifier. For list of available locations, refer to getLocations operation. |
| minMembers | integer | query | Filter by minimum number of clan members |
| maxMembers | integer | query | Filter by maximum number of clan members |
| minClanPoints | integer | query | Filter by minimum amount of clan points. |
| minClanLevel | integer | query | Filter by minimum clan level. |
| limit | integer | query | Limit the number of items returned in the response. |
| after | string | query | Return only items that occur after this marker. Before marker can be found from the response, inside the 'paging' property. Note that only after or before can be specified for a request, not both. |
| before | string | query | Return only items that occur before this marker. Before marker can be found from the response, inside the 'paging' property. Note that only after or before can be specified for a request, not both. |
| labelIds | string | query | Comma separated list of label IDs to use for filtering results. |


**Obtain the JSON code and store it in the file clans.json.**

In [1]:
import time
import math
import json
import requests
import logging
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from ratelimit import limits, sleep_and_retry
from tqdm import tqdm

# Extract Clans Tags

In [2]:
file_path = 'others/clans.json'

# Function to open the file and extract tags, specifying the encoding
def extract_tags_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:  # Specifying the encoding here
        data = json.load(file)
        return [item.get("tag") for item in data.get("items", [])]

# Extract tags from the specified file
try:
    extracted_tags = extract_tags_from_file(file_path)
    print(extracted_tags)
    
except UnicodeDecodeError as e:
    print(f"Error reading the file: {e}")

['#2QCVGYYRY', '#2YVUJJ00J', '#20G90JGLQ', '#V00PUJ8V', '#LY8U92LY', '#C8JLLGJ8', '#YQQJGVJL', '#Q8P8QVV0', '#2PVPCQ8YL', '#2PC0LQ09L', '#2GCRJ0L9Q', '#2GPVJ9V8J', '#GRPPJYY0', '#29L92L090', '#V2UQ808R', '#R0Q8RQRC', '#RVV9CQU0', '#YV9QGCLL', '#2Q8RU998Y', '#CRUPJGRY', '#2L8CCUQYV', '#2G0CV29VP', '#QR0JYRR0', '#J0GV8UV9', '#2QC8QVJUQ', '#299LP0YJU', '#2RYQY288', '#2G8VL2JJP', '#9V929YCJ', '#QYULJ2GL', '#2PR92YGG0', '#GPG2VQR8', '#LQ8PYLYC', '#LVCP2RR0', '#GP8QY2GU', '#22CYLRVUC', '#2G9GJ8Q0C', '#YR8QU0JY', '#2Q8LUCV8U', '#290JPLQ0U', '#P00PUU28', '#PQ2CCJL', '#JRRGU2P0', '#28208L2JU', '#2QLURQR9J', '#8L22PR0V', '#GVYJU2GR', '#JLQLPGPL', '#90J0P8JC', '#2RRY020PC', '#2QJUJPY9J', '#2QP9QLLVV', '#2PRV8RY82', '#2YUQJ8UQP', '#2QYUPJQ9R', '#202LVPR0V', '#RGR909GQ', '#RUG299L9', '#2RVJU80LY', '#UR9GYCL', '#2G00J9UPR', '#LJPLQGC0', '#C8JVR090', '#QVLCU20Q', '#GQR09YC', '#JURYYGPJ', '#2LCYYUU2G', '#U8Y8GU2Y', '#2Q8CJYGPU', '#VU0UQ8QV', '#2GL9L8RL2', '#2QJYLGVVC', '#2GYV992LU', '#2G09JPJPV', '#20

In [3]:
# the extracted clan tags have '#' in beigning we have to replace it with URL encode '%23'
def update_tags(extracted_tags):
    # Replace '#' with '%23' for each tag in the list
    updated_tags = [tag.replace('#', '%23') for tag in extracted_tags]
    return updated_tags

# Get the updated list of tags
updated_extracted_tags = update_tags(extracted_tags)

# Print or return the updated list
print(updated_extracted_tags)

['%232QCVGYYRY', '%232YVUJJ00J', '%2320G90JGLQ', '%23V00PUJ8V', '%23LY8U92LY', '%23C8JLLGJ8', '%23YQQJGVJL', '%23Q8P8QVV0', '%232PVPCQ8YL', '%232PC0LQ09L', '%232GCRJ0L9Q', '%232GPVJ9V8J', '%23GRPPJYY0', '%2329L92L090', '%23V2UQ808R', '%23R0Q8RQRC', '%23RVV9CQU0', '%23YV9QGCLL', '%232Q8RU998Y', '%23CRUPJGRY', '%232L8CCUQYV', '%232G0CV29VP', '%23QR0JYRR0', '%23J0GV8UV9', '%232QC8QVJUQ', '%23299LP0YJU', '%232RYQY288', '%232G8VL2JJP', '%239V929YCJ', '%23QYULJ2GL', '%232PR92YGG0', '%23GPG2VQR8', '%23LQ8PYLYC', '%23LVCP2RR0', '%23GP8QY2GU', '%2322CYLRVUC', '%232G9GJ8Q0C', '%23YR8QU0JY', '%232Q8LUCV8U', '%23290JPLQ0U', '%23P00PUU28', '%23PQ2CCJL', '%23JRRGU2P0', '%2328208L2JU', '%232QLURQR9J', '%238L22PR0V', '%23GVYJU2GR', '%23JLQLPGPL', '%2390J0P8JC', '%232RRY020PC', '%232QJUJPY9J', '%232QP9QLLVV', '%232PRV8RY82', '%232YUQJ8UQP', '%232QYUPJQ9R', '%23202LVPR0V', '%23RGR909GQ', '%23RUG299L9', '%232RVJU80LY', '%23UR9GYCL', '%232G00J9UPR', '%23LJPLQGC0', '%23C8JVR090', '%23QVLCU20Q', '%23GQR09YC

# Extract Players Tags

In [4]:
# Replace your API_KEY
api_key = ''  

# Base URL for the Clash of Clans API clans endpoint
base_url = 'https://api.clashofclans.com/v1/clans/'

# Header to include in the request
headers = {
   'Authorization': f'Bearer {api_key}',
   'Accept': 'application/json'
}

# Function to get clan member list for each clan tag
def get_clan_members(clan_tags):
    clan_members = {}  # Dictionary to store clan members list by clan tag
   
    for tag in clan_tags:
        # Constructing the full URL for the clan members endpoint
        full_url = f'{base_url}{tag}/members'
        response = requests.get(full_url, headers=headers)
        
        if response.status_code == 200:
            # Successful response
            data = response.json()
            # Assuming the API returns a list of clan members directly
            clan_members[tag] = data.get('items', [])
            
        else:
            # Handle errors or unsuccessful responses
            print(f'Failed to fetch clan members for {tag}: HTTP {response.status_code}')
            
    return clan_members

# Get clan members for each tag
clan_members_lists = get_clan_members(updated_extracted_tags)

# Example: print the result for the first clan
first_tag = updated_extracted_tags[0]
print(f'Clan members for {first_tag}:', clan_members_lists[first_tag])

Clan members for %232QCVGYYRY: [{'tag': '#9Y8LQULQC', 'name': 'VE.HELL FORCES', 'role': 'coLeader', 'townHallLevel': 13, 'expLevel': 169, 'league': {'id': 29000000, 'name': 'Unranked', 'iconUrls': {'small': 'https://api-assets.clashofclans.com/leagues/72/e--YMyIexEQQhE4imLoJcwhYn6Uy8KqlgyY3_kFV6t4.png', 'tiny': 'https://api-assets.clashofclans.com/leagues/36/e--YMyIexEQQhE4imLoJcwhYn6Uy8KqlgyY3_kFV6t4.png'}}, 'trophies': 3073, 'builderBaseTrophies': 1954, 'clanRank': 1, 'previousClanRank': 1, 'donations': 265, 'donationsReceived': 0, 'playerHouse': {'elements': [{'type': 'ground', 'id': 82000002}, {'type': 'walls', 'id': 82000049}, {'type': 'roof', 'id': 82000017}, {'type': 'decoration', 'id': 82000062}]}, 'builderBaseLeague': {'id': 44000019, 'name': 'Copper League I'}}, {'tag': '#QJV29VQ02', 'name': 'kafeel4', 'role': 'coLeader', 'townHallLevel': 12, 'expLevel': 102, 'league': {'id': 29000013, 'name': 'Master League III', 'iconUrls': {'small': 'https://api-assets.clashofclans.com/lea

# Convert the data to a pandas DataFrame

In [5]:
# Assuming clan_members_lists is your dictionary from the modified get_clan_members function
def convert_to_dataframe(clan_members_lists):
# Create a list of tuples (clan_tag, player_tag) for all clans
    data = [(clan_tag, player_tag) for clan_tag, player_tags in clan_members_lists.items() for player_tag in player_tags]
    
# Convert the list of tuples into a DataFrame
    df = pd.DataFrame(data, columns=['Clan Tag', 'Player Tag'])
    
    return df

# Convert the dictionary to a DataFrame
df_clan_members = convert_to_dataframe(clan_members_lists)
print(df_clan_members)

           Clan Tag                                         Player Tag
0      %232QCVGYYRY  {'tag': '#9Y8LQULQC', 'name': 'VE.HELL FORCES'...
1      %232QCVGYYRY  {'tag': '#QJV29VQ02', 'name': 'kafeel4', 'role...
2      %232QCVGYYRY  {'tag': '#QJVGGV8R0', 'name': 'N-R', 'role': '...
3      %232QCVGYYRY  {'tag': '#L98YLG8Y8', 'name': 'Guz Rama', 'rol...
4      %232QCVGYYRY  {'tag': '#QJJU8G2RR', 'name': 'BoiledRavioli',...
...             ...                                                ...
41594  %232QLUQ8J98  {'tag': '#QUQ0QQ9UL', 'name': 'EXODUS⚔️08', 'r...
41595  %232QLUQ8J98  {'tag': '#GLJGQYQPQ', 'name': 'TricKy', 'role'...
41596  %232QLUQ8J98  {'tag': '#QCCPUJGPU', 'name': 'MīñīmīlīTīā', '...
41597  %232QLUQ8J98  {'tag': '#GYU0CJ8LP', 'name': 'YUNO√', 'role':...
41598  %232QLUQ8J98  {'tag': '#G9JR8VL9G', 'name': 'skupido', 'role...

[41599 rows x 2 columns]


In [6]:
def convert_to_dataframe(clan_members_lists):
    # Initialize an empty list to store the data
    data = []
    
    # Loop through each clan tag and its corresponding list of members
    for clan_tag, members in clan_members_lists.items():
        for member in members:
            # For each member, extract the clan tag and the player tag, ensuring the player tag is a string
            player_tag = member['tag']  # Assuming 'tag' key exists and its value is the player's tag
            data.append((clan_tag, player_tag))
    
    # Convert the list of tuples into a DataFrame
    df = pd.DataFrame(data, columns=['Clan Tag', 'Player Tag'])
    
    # Optional: Convert clan and player tags to ensure they are URL-friendly
    # This step is optional and depends on whether you need to use these tags in URLs
    df['Clan Tag'] = df['Clan Tag'].apply(lambda x: x.replace('%23', '#'))
    # df['Player Tag'] = df['Player Tag'].apply(lambda x: x.replace('#', '%23'))
    
    return df

# Example usage with your clan_members_lists dictionary
# Make sure to replace 'clan_members_lists' with your actual dictionary variable
df_clan_members = convert_to_dataframe(clan_members_lists)
print(df_clan_members)

df_clan_members.to_csv('datasets/clan_and_player_tags.csv', index=False)

         Clan Tag  Player Tag
0      #2QCVGYYRY  #9Y8LQULQC
1      #2QCVGYYRY  #QJV29VQ02
2      #2QCVGYYRY  #QJVGGV8R0
3      #2QCVGYYRY  #L98YLG8Y8
4      #2QCVGYYRY  #QJJU8G2RR
...           ...         ...
41594  #2QLUQ8J98  #QUQ0QQ9UL
41595  #2QLUQ8J98  #GLJGQYQPQ
41596  #2QLUQ8J98  #QCCPUJGPU
41597  #2QLUQ8J98  #GYU0CJ8LP
41598  #2QLUQ8J98  #G9JR8VL9G

[41599 rows x 2 columns]


# Extract Players Information

In [7]:
# Set up logging to file
logging.basicConfig(filename='others/error_log.log', level=logging.ERROR, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Load the CSV file into a DataFrame
df = pd.read_csv('datasets/clan_and_player_tags.csv')  # Update this path to your actual CSV file location

# API details
api_key = ''
headers = {'Authorization': 'Bearer ' + api_key}

# Rate limit: 10 requests per second (adjust as needed)
@sleep_and_retry
@limits(calls=10, period=1)
def call_api(url):
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

def fetch_player_details(tag):
    url = f'https://api.clashofclans.com/v1/players/{tag.replace("#", "%23")}'
    try:
        data = call_api(url)
        return {
            'name': data.get('name', ''),
            'role': data.get('role', ''),
            'league': data.get('league', {}).get('name', ''),
            'builderBaseLeague': data.get('builderBaseLeague', {}).get('name', ''),
            'townHallLevel': data.get('townHallLevel', 0),
            'builderHallLevel': data.get('builderHallLevel', 0),
            'expLevel': data.get('expLevel', 0),
            'trophies': data.get('trophies', 0),
            'bestTrophies': data.get('bestTrophies', 0),
            'builderBaseTrophies': data.get('builderBaseTrophies', 0),
            'bestBuilderBaseTrophies': data.get('bestBuilderBaseTrophies', 0),
            'attackWins': data.get('attackWins', 0),
            'defenseWins': data.get('defenseWins', 0),
            'warStars': data.get('warStars', 0),
            'clanCapitalContributions': data.get('clanCapitalContributions', 0),
            'donations': data.get('donations', 0),
            'donationsReceived': data.get('donationsReceived', 0),
        }
    except requests.exceptions.RequestException as e:
        logging.error(f'Error fetching data for {tag}: {e}')

def process_batch(batch):
    player_details = []
    with ThreadPoolExecutor(max_workers=20) as executor:
        future_to_tag = {executor.submit(fetch_player_details, tag): tag for tag in batch}
        for future in as_completed(future_to_tag):
            tag = future_to_tag[future]
            try:
                details = future.result()
                player_details.append(details)
            except Exception as exc:
                logging.error(f'{tag} generated an exception: {exc}')
    
    return player_details

def save_checkpoint(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f)

def load_checkpoint(filename):
    try:
        with open(filename, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        return []

def main():
    batch_size = 1000
    checkpoint_file = 'others/player_details_checkpoint.json'
    
    # Load checkpoint if it exists
    player_details = load_checkpoint(checkpoint_file)
    start_index = len(player_details)

    num_batches = math.ceil((len(df) - start_index) / batch_size)
    
    for i in tqdm(range(start_index, len(df), batch_size), total=num_batches, desc="Processing batches"):
        batch = df['Player Tag'].iloc[i:i+batch_size].tolist()
        batch_details = process_batch(batch)
        player_details.extend(batch_details)
        
        # Save checkpoint after each batch
        save_checkpoint(player_details, checkpoint_file)

     # Filter out None values from player_details
    player_details = [detail for detail in player_details if detail is not None]
    
    # Create a new DataFrame with the player details
    details_df = pd.DataFrame(player_details)

    # Merge the original DataFrame with the new details DataFrame
    final_df = pd.concat([df, details_df], axis=1)

    # Save the final DataFrame to a new CSV file
    final_df.to_csv('datasets/player_details.csv', index=False)
    print("Data fetching and processing complete. Results saved to 'Clan_and_Player_Details.csv'")

if __name__ == "__main__":
    start_time = time.time()
    main()
    end_time = time.time()
    print(f"Total execution time: {end_time - start_time:.2f} seconds")

Processing batches: 100%|██████████| 42/42 [1:09:58<00:00, 99.96s/it] 


Data fetching and processing complete. Results saved to 'Clan_and_Player_Details.csv'
Total execution time: 4199.10 seconds
